In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer 
%matplotlib inline
from sklearn.feature_extraction import text as TEXT
import re
from nltk.tokenize import RegexpTokenizer,wordpunct_tokenize, sent_tokenize
import os
from gensim.models import ldamulticore
from nltk.stem import WordNetLemmatizer
from gensim import corpora, models, similarities, matutils
from sklearn.decomposition import TruncatedSVD
from nltk.tokenize import RegexpTokenizer
from nltk.stem import LancasterStemmer, PorterStemmer
from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
import nltk
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# Work cells

In [2]:
import glob
import os

#file_list = glob.glob(os.path.join("d:/downloads/cnn/sub_st/", "*.story",))
file_list = glob.glob(os.path.join("/home/ubuntu/cnn/", "*.story",))
corpus = {}

for i,file_path in enumerate(file_list):
    with open(file_path,encoding="utf8") as f_input:
        corpus[i]=f_input.read().strip("\n")
        #print(i)
# print(corpus)  

In [3]:
lemmer=WordNetLemmatizer()
stemmer = PorterStemmer()
regex=RegexpTokenizer("\\b[a-zA-Z][a-zA-Z][a-zA-Z]+\\b")

def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        
        stemmed.append(lemmer.lemmatize(item,"v"))
    return stemmed

def tokenize(text):
    tokens = regex.tokenize(text)
    stems = stem_tokens(tokens, stemmer)
    return stems
stop=["(CNN)",'com',"people","new","says","said","time","like","just","years","student news",
      "news","cnn student",'cnn',"wa","ha","highlight","say","want","know","according","tell",
      "come","way","lot","try","look","really","use","feel","make","year"]

stop_wd=TEXT.ENGLISH_STOP_WORDS.union(stop)

# Create a CountVectorizer for parsing/counting words

In [9]:
count_vectorizer = CountVectorizer(ngram_range=(1, 2),tokenizer=tokenize,  
                                   stop_words=stop_wd,min_df=10,max_df=0.8)
mat_cv=count_vectorizer.fit_transform(corpus.values())
mat_cv.shape

(92579, 423824)

In [79]:
#pickling the processed Count Vectorizer
import pickle
pickling_on = open("mat_cv.pickle","wb")
pickle.dump(mat_cv, pickling_on)
pickling_on.close()


In [4]:
#Loading pickled data as Count Vectorizer
import pickle
pickle_off = open("mat_cv.pickle","rb")
mat_cv = pickle.load(pickle_off)

# Using LDA Model

In [6]:
# Convert sparse matrix of counts to a gensim corpus for LDA Model

cv_2corpus = matutils.Sparse2Corpus(mat_cv.transpose())
# ld_words2idf=dict((v,k) for k,v in count_vectorizer.vocabulary_.items())
# ld_words2idf=corpora.Dictionary.from_corpus(corpus=cv_2corpus,id2word=ld_words2idf)

# pickling_on = open("ld_words2idf.pkl","wb")
# pickle.dump(ld_words2idf, pickling_on)
# pickling_on.close()

In [5]:
pickle_off = open("ld_words2idf.pkl","rb")
ld_words2idf = pickle.load(pickle_off)

In [35]:
# Using LDA Model
logging.basicConfig(filename='~/gensim.log',
                    format="%(asctime)s:%(levelname)s:%(message)s",
                    level=logging.INFO)

lda=ldamulticore.LdaMulticore(cv_2corpus,id2word=ld_words2idf,num_topics=33,passes=50,eval_every=None,workers=12,
                              chunksize=10000)

2018-03-03 17:22:53,229 : INFO : using symmetric alpha at 0.030303030303030304
2018-03-03 17:22:53,230 : INFO : using symmetric eta at 0.030303030303030304
2018-03-03 17:22:53,279 : INFO : using serial LDA version on this node
2018-03-03 17:25:33,323 : INFO : running online LDA training, 33 topics, 50 passes over the supplied corpus of 92579 documents, updating every 120000 documents, evaluating every ~0 documents, iterating 50x with a convergence threshold of 0.001000
2018-03-03 17:25:33,324 : INFO : training LDA model using 12 processes
2018-03-03 17:25:34,438 : INFO : PROGRESS: pass 0, dispatched chunk #0 = documents up to #10000/92579, outstanding queue size 1
2018-03-03 17:25:53,969 : INFO : PROGRESS: pass 0, dispatched chunk #1 = documents up to #20000/92579, outstanding queue size 2
2018-03-03 17:26:13,509 : INFO : PROGRESS: pass 0, dispatched chunk #2 = documents up to #30000/92579, outstanding queue size 3
2018-03-03 17:26:33,039 : INFO : PROGRESS: pass 0, dispatched chunk #3 

2018-03-03 17:41:01,069 : INFO : PROGRESS: pass 3, dispatched chunk #3 = documents up to #40000/92579, outstanding queue size 3
2018-03-03 17:41:21,686 : INFO : PROGRESS: pass 3, dispatched chunk #4 = documents up to #50000/92579, outstanding queue size 3
2018-03-03 17:41:41,179 : INFO : PROGRESS: pass 3, dispatched chunk #5 = documents up to #60000/92579, outstanding queue size 3
2018-03-03 17:42:00,961 : INFO : PROGRESS: pass 3, dispatched chunk #6 = documents up to #70000/92579, outstanding queue size 3
2018-03-03 17:42:20,739 : INFO : PROGRESS: pass 3, dispatched chunk #7 = documents up to #80000/92579, outstanding queue size 3
2018-03-03 17:42:40,321 : INFO : PROGRESS: pass 3, dispatched chunk #8 = documents up to #90000/92579, outstanding queue size 3
2018-03-03 17:42:59,578 : INFO : PROGRESS: pass 3, dispatched chunk #9 = documents up to #92579/92579, outstanding queue size 3
2018-03-03 17:44:24,952 : INFO : topic #27 (0.030): 0.002*"cancer" + 0.002*"think" + 0.002*"police" + 0.

2018-03-03 17:57:02,313 : INFO : topic #12 (0.030): 0.008*"flight" + 0.006*"oil" + 0.006*"plane" + 0.005*"air" + 0.004*"pilot" + 0.004*"crash" + 0.004*"aircraft" + 0.003*"fly" + 0.003*"report" + 0.003*"airlines"
2018-03-03 17:57:02,320 : INFO : topic #32 (0.030): 0.005*"children" + 0.003*"state" + 0.003*"women" + 0.003*"life" + 0.003*"report" + 0.003*"right" + 0.003*"live" + 0.002*"child" + 0.002*"care" + 0.002*"family"
2018-03-03 17:57:02,326 : INFO : topic #29 (0.030): 0.006*"storm" + 0.005*"water" + 0.005*"state" + 0.004*"flood" + 0.004*"city" + 0.004*"report" + 0.004*"china" + 0.003*"weather" + 0.003*"home" + 0.003*"wind"
2018-03-03 17:57:02,331 : INFO : topic #6 (0.030): 0.004*"jackson" + 0.004*"state" + 0.004*"syria" + 0.004*"president" + 0.003*"iran" + 0.003*"military" + 0.003*"egypt" + 0.003*"war" + 0.002*"government" + 0.002*"obama"
2018-03-03 17:57:02,355 : INFO : topic diff=1.236172, rho=0.248009
2018-03-03 17:57:03,672 : INFO : PROGRESS: pass 7, dispatched chunk #0 = docume

2018-03-03 18:09:17,317 : INFO : PROGRESS: pass 10, dispatched chunk #0 = documents up to #10000/92579, outstanding queue size 1
2018-03-03 18:09:36,688 : INFO : PROGRESS: pass 10, dispatched chunk #1 = documents up to #20000/92579, outstanding queue size 2
2018-03-03 18:09:56,124 : INFO : PROGRESS: pass 10, dispatched chunk #2 = documents up to #30000/92579, outstanding queue size 3
2018-03-03 18:10:31,508 : INFO : PROGRESS: pass 10, dispatched chunk #3 = documents up to #40000/92579, outstanding queue size 3
2018-03-03 18:10:58,946 : INFO : PROGRESS: pass 10, dispatched chunk #4 = documents up to #50000/92579, outstanding queue size 3
2018-03-03 18:11:19,494 : INFO : PROGRESS: pass 10, dispatched chunk #5 = documents up to #60000/92579, outstanding queue size 3
2018-03-03 18:11:39,148 : INFO : PROGRESS: pass 10, dispatched chunk #6 = documents up to #70000/92579, outstanding queue size 3
2018-03-03 18:11:58,790 : INFO : PROGRESS: pass 10, dispatched chunk #7 = documents up to #80000/

2018-03-03 18:23:57,578 : INFO : PROGRESS: pass 13, dispatched chunk #7 = documents up to #80000/92579, outstanding queue size 3
2018-03-03 18:24:17,174 : INFO : PROGRESS: pass 13, dispatched chunk #8 = documents up to #90000/92579, outstanding queue size 3
2018-03-03 18:24:36,923 : INFO : PROGRESS: pass 13, dispatched chunk #9 = documents up to #92579/92579, outstanding queue size 3
2018-03-03 18:25:36,400 : INFO : topic #0 (0.030): 0.010*"women" + 0.005*"music" + 0.005*"space" + 0.004*"work" + 0.003*"think" + 0.003*"world" + 0.003*"life" + 0.003*"help" + 0.003*"record" + 0.003*"play"
2018-03-03 18:25:36,406 : INFO : topic #7 (0.030): 0.006*"obama" + 0.006*"state" + 0.006*"president" + 0.005*"vote" + 0.005*"house" + 0.004*"republican" + 0.004*"party" + 0.004*"republicans" + 0.004*"senate" + 0.003*"democrats"
2018-03-03 18:25:36,411 : INFO : topic #32 (0.030): 0.009*"children" + 0.006*"women" + 0.004*"child" + 0.004*"parent" + 0.004*"state" + 0.004*"right" + 0.003*"care" + 0.003*"live"

...

2018-03-03 19:43:10,240 : INFO : topic #15 (0.030): 0.014*"africa" + 0.010*"world" + 0.008*"health" + 0.008*"ebola" + 0.008*"aid" + 0.007*"african" + 0.007*"help" + 0.006*"countries" + 0.006*"virus" + 0.005*"haiti"
2018-03-03 19:43:10,246 : INFO : topic #28 (0.030): 0.026*"church" + 0.012*"pope" + 0.010*"religious" + 0.008*"catholic" + 0.006*"christian" + 0.006*"vatican" + 0.006*"god" + 0.005*"francis" + 0.005*"religion" + 0.005*"faith"
2018-03-03 19:43:10,270 : INFO : topic diff=0.169014, rho=0.152043
2018-03-03 19:43:10,718 : INFO : PROGRESS: pass 34, dispatched chunk #0 = documents up to #10000/92579, outstanding queue size 1
2018-03-03 19:43:29,940 : INFO : PROGRESS: pass 34, dispatched chunk #1 = documents up to #20000/92579, outstanding queue size 2
2018-03-03 19:43:50,414 : INFO : PROGRESS: pass 34, dispatched chunk #2 = documents up to #30000/92579, outstanding queue size 3
2018-03-03 19:44:10,080 : INFO : PROGRESS: pass 34, dispatched chunk #3 = documents up to #40000/92579, o

2018-03-03 19:55:37,798 : INFO : PROGRESS: pass 37, dispatched chunk #3 = documents up to #40000/92579, outstanding queue size 3
2018-03-03 19:55:57,704 : INFO : PROGRESS: pass 37, dispatched chunk #4 = documents up to #50000/92579, outstanding queue size 3
2018-03-03 19:56:17,117 : INFO : PROGRESS: pass 37, dispatched chunk #5 = documents up to #60000/92579, outstanding queue size 3
2018-03-03 19:56:36,831 : INFO : PROGRESS: pass 37, dispatched chunk #6 = documents up to #70000/92579, outstanding queue size 3
2018-03-03 19:56:57,374 : INFO : PROGRESS: pass 37, dispatched chunk #7 = documents up to #80000/92579, outstanding queue size 3
2018-03-03 19:57:16,934 : INFO : PROGRESS: pass 37, dispatched chunk #8 = documents up to #90000/92579, outstanding queue size 3
2018-03-03 19:57:36,208 : INFO : PROGRESS: pass 37, dispatched chunk #9 = documents up to #92579/92579, outstanding queue size 3
2018-03-03 19:58:27,531 : INFO : topic #11 (0.030): 0.033*"obama" + 0.023*"president" + 0.008*"cl

2018-03-03 20:10:41,791 : INFO : topic #22 (0.030): 0.016*"israel" + 0.011*"israeli" + 0.009*"iran" + 0.007*"india" + 0.007*"palestinian" + 0.007*"gaza" + 0.006*"gadhafi" + 0.005*"state" + 0.005*"hamas" + 0.005*"libya"
2018-03-03 20:10:41,798 : INFO : topic #3 (0.030): 0.018*"film" + 0.010*"gun" + 0.009*"movie" + 0.008*"star" + 0.005*"best" + 0.005*"award" + 0.005*"actor" + 0.004*"shoot" + 0.004*"play" + 0.004*"character"
2018-03-03 20:10:41,804 : INFO : topic #23 (0.030): 0.007*"state" + 0.006*"government" + 0.005*"law" + 0.004*"pay" + 0.004*"company" + 0.004*"economic" + 0.004*"money" + 0.003*"billion" + 0.003*"bank" + 0.003*"million"
2018-03-03 20:10:41,810 : INFO : topic #12 (0.030): 0.016*"flight" + 0.010*"plane" + 0.009*"air" + 0.009*"oil" + 0.008*"crash" + 0.007*"airport" + 0.007*"pilot" + 0.007*"fly" + 0.006*"aircraft" + 0.006*"passengers"
2018-03-03 20:10:41,815 : INFO : topic #2 (0.030): 0.007*"city" + 0.005*"park" + 0.004*"hotel" + 0.004*"travel" + 0.004*"build" + 0.003*"wor

2018-03-03 20:22:09,300 : INFO : topic #26 (0.030): 0.032*"police" + 0.008*"officer" + 0.007*"shoot" + 0.007*"authorities" + 0.006*"report" + 0.006*"arrest" + 0.005*"man" + 0.005*"accord" + 0.004*"county" + 0.004*"suspect"
2018-03-03 20:22:09,323 : INFO : topic diff=0.077867, rho=0.137028
2018-03-03 20:22:09,773 : INFO : PROGRESS: pass 44, dispatched chunk #0 = documents up to #10000/92579, outstanding queue size 1
2018-03-03 20:22:28,959 : INFO : PROGRESS: pass 44, dispatched chunk #1 = documents up to #20000/92579, outstanding queue size 2
2018-03-03 20:22:48,370 : INFO : PROGRESS: pass 44, dispatched chunk #2 = documents up to #30000/92579, outstanding queue size 3
2018-03-03 20:23:07,960 : INFO : PROGRESS: pass 44, dispatched chunk #3 = documents up to #40000/92579, outstanding queue size 3
2018-03-03 20:23:27,678 : INFO : PROGRESS: pass 44, dispatched chunk #4 = documents up to #50000/92579, outstanding queue size 3
2018-03-03 20:23:47,185 : INFO : PROGRESS: pass 44, dispatched ch

2018-03-03 20:35:13,888 : INFO : PROGRESS: pass 47, dispatched chunk #5 = documents up to #60000/92579, outstanding queue size 3
2018-03-03 20:35:33,667 : INFO : PROGRESS: pass 47, dispatched chunk #6 = documents up to #70000/92579, outstanding queue size 3
2018-03-03 20:35:53,347 : INFO : PROGRESS: pass 47, dispatched chunk #7 = documents up to #80000/92579, outstanding queue size 3
2018-03-03 20:36:13,271 : INFO : PROGRESS: pass 47, dispatched chunk #8 = documents up to #90000/92579, outstanding queue size 3
2018-03-03 20:36:32,583 : INFO : PROGRESS: pass 47, dispatched chunk #9 = documents up to #92579/92579, outstanding queue size 3
2018-03-03 20:37:23,475 : INFO : topic #2 (0.030): 0.007*"city" + 0.005*"park" + 0.004*"hotel" + 0.004*"travel" + 0.004*"build" + 0.003*"world" + 0.003*"place" + 0.003*"room" + 0.003*"day" + 0.003*"home"
2018-03-03 20:37:23,481 : INFO : topic #31 (0.030): 0.008*"report" + 0.005*"department" + 0.005*"investigation" + 0.005*"state" + 0.004*"police" + 0.00

In [7]:
#lets pickle the model
import pickle
pickling_on = open("lda_model_fri.pkl","wb")
pickle.dump(lda, pickling_on)
pickling_on.close()
lda.save('lda.model')

#loading the pickled model
pickle_off = open("lda_model_fri.pkl","rb")
lda = pickle.load(pickle_off)

In [37]:
lda.print_topics(num_words=15,num_topics=33)

2018-03-03 20:45:31,649 : INFO : topic #0 (0.030): 0.009*"music" + 0.007*"space" + 0.005*"women" + 0.004*"band" + 0.004*"record" + 0.004*"nasa" + 0.004*"album" + 0.004*"work" + 0.004*"song" + 0.003*"weight" + 0.003*"play" + 0.003*"think" + 0.003*"earth" + 0.003*"life" + 0.003*"rock"
2018-03-03 20:45:31,656 : INFO : topic #1 (0.030): 0.018*"russia" + 0.016*"russian" + 0.011*"ukraine" + 0.009*"win" + 0.008*"open" + 0.007*"putin" + 0.007*"world" + 0.006*"set" + 0.006*"match" + 0.006*"play" + 0.005*"moscow" + 0.005*"seed" + 0.005*"beat" + 0.004*"tennis" + 0.004*"final"
2018-03-03 20:45:31,662 : INFO : topic #2 (0.030): 0.007*"city" + 0.005*"park" + 0.004*"hotel" + 0.004*"travel" + 0.004*"build" + 0.003*"place" + 0.003*"world" + 0.003*"room" + 0.003*"day" + 0.003*"home" + 0.003*"include" + 0.003*"offer" + 0.003*"best" + 0.003*"open" + 0.002*"house"
2018-03-03 20:45:31,666 : INFO : topic #3 (0.030): 0.018*"film" + 0.010*"gun" + 0.010*"movie" + 0.008*"star" + 0.005*"best" + 0.005*"actor" + 0.

2018-03-03 20:45:31,794 : INFO : topic #28 (0.030): 0.026*"church" + 0.012*"pope" + 0.011*"religious" + 0.008*"catholic" + 0.007*"christian" + 0.006*"god" + 0.006*"religion" + 0.006*"vatican" + 0.005*"faith" + 0.005*"francis" + 0.004*"jesus" + 0.004*"shark" + 0.004*"jewish" + 0.004*"world" + 0.004*"argentina"
2018-03-03 20:45:31,797 : INFO : topic #29 (0.030): 0.009*"storm" + 0.007*"water" + 0.007*"state" + 0.006*"flood" + 0.006*"city" + 0.006*"weather" + 0.005*"report" + 0.005*"home" + 0.005*"wind" + 0.004*"residents" + 0.004*"snow" + 0.004*"rain" + 0.004*"national" + 0.004*"service" + 0.003*"area"
2018-03-03 20:45:31,800 : INFO : topic #30 (0.030): 0.012*"government" + 0.009*"president" + 0.008*"country" + 0.008*"protest" + 0.008*"minister" + 0.007*"party" + 0.007*"political" + 0.005*"protesters" + 0.005*"prime" + 0.005*"prime minister" + 0.004*"right" + 0.004*"election" + 0.004*"vote" + 0.004*"state" + 0.003*"power"
2018-03-03 20:45:31,807 : INFO : topic #31 (0.030): 0.008*"report" 

[(0,
  '0.009*"music" + 0.007*"space" + 0.005*"women" + 0.004*"band" + 0.004*"record" + 0.004*"nasa" + 0.004*"album" + 0.004*"work" + 0.004*"song" + 0.003*"weight" + 0.003*"play" + 0.003*"think" + 0.003*"earth" + 0.003*"life" + 0.003*"rock"'),
 (1,
  '0.018*"russia" + 0.016*"russian" + 0.011*"ukraine" + 0.009*"win" + 0.008*"open" + 0.007*"putin" + 0.007*"world" + 0.006*"set" + 0.006*"match" + 0.006*"play" + 0.005*"moscow" + 0.005*"seed" + 0.005*"beat" + 0.004*"tennis" + 0.004*"final"'),
 (2,
  '0.007*"city" + 0.005*"park" + 0.004*"hotel" + 0.004*"travel" + 0.004*"build" + 0.003*"place" + 0.003*"world" + 0.003*"room" + 0.003*"day" + 0.003*"home" + 0.003*"include" + 0.003*"offer" + 0.003*"best" + 0.003*"open" + 0.002*"house"'),
 (3,
  '0.018*"film" + 0.010*"gun" + 0.010*"movie" + 0.008*"star" + 0.005*"best" + 0.005*"actor" + 0.005*"award" + 0.004*"shoot" + 0.004*"play" + 0.004*"character" + 0.004*"series" + 0.004*"hollywood" + 0.004*"win" + 0.003*"zimmerman" + 0.003*"million"'),
 (4,
  '

In [8]:
# Transform the docs from the word space to the topic space (like "transform" in sklearn)
lda_corpus = lda[cv_2corpus]
lda_corpus


In [9]:
# Store the documents' topic vectors in a list so we can take a peak
lda_docs = [doc for doc in lda_corpus]

In [8]:
#pickle lda_docs

import pickle
pickling_on = open("lda_docs.pickle","wb")
pickle.dump(lda_docs, pickling_on)
pickling_on.close()



pickle_off = open("lda_docs.pickle","rb")
lda_docs = pickle.load(pickle_off)


In [9]:
print("Topic Values", lda_docs[0]) # print Topic Values for document 0 

Topic Values [(7, 0.41884512), (9, 0.011072788), (12, 0.013776924), (18, 0.087512776), (23, 0.30254132), (25, 0.023175688), (26, 0.028242573), (28, 0.05824326), (32, 0.052103791)]


In [10]:
lda.num_topics

33

In [9]:

#consolidated data frame with document number , topic_number, and highest topic score and related topic terms
number_of_topics=33
topics=lda.show_topics(num_topics=number_of_topics)
topic_df=pd.DataFrame.from_records(topics,columns=["topic_id","topic_terms"])

highest_score_topic_lda_docs=[sorted(i,key=lambda x: x[1],reverse=True)[0] for i in lda_docs]

topics_by_documnent_numbers=pd.DataFrame(data=highest_score_topic_lda_docs,columns=["topic_number","topic_score"])

topics_by_documnent_numbers=topics_by_documnent_numbers.join(topic_df,on='topic_number')
topics_by_documnent_numbers.drop("topic_id",axis=1,inplace=True)

docnument_number=topics_by_documnent_numbers.reset_index()

topics_by_documnent_numbers=topics_by_documnent_numbers.reset_index()

topics_by_documnent_numbers.rename(columns={"index":"document_number"},inplace=True)

In [18]:
#topics_by_documnent_numbers.head(2)

,document_number,topic_number,topic_score,topic_terms
0,0,7,0.418845,"0.007*""vote"" + 0.006*""state"" + 0.006*""republic..."
1,1,23,0.512726,"0.006*""government"" + 0.006*""state"" + 0.005*""la..."


In [43]:
lda.get_document_topics()
#lda.get_document_topics(ld_words2idf)

TypeError: 'Dictionary' object is not callable

In [10]:
import seaborn as sns
docs_per_topic=(topics_by_documnent_numbers.topic_number.value_counts())

In [11]:
docs_per_topic.to_csv("docs_per_topic.csv")

In [13]:
#prints text and topic_terms
def get_document2topic(doc_number):
    
    return(topics_by_documnent_numbers["topic_terms"].iloc[doc_number],corpus[doc_number][:2000])

topic, text=get_document2topic(9000)
print("\n"+"\033[1m")
print("Topic_terms:" ,(tokenize(topic)),"\n")
print("Document",text,"...")



Topic_terms: ['team', 'win', 'game', 'league', 'club', 'cup', 'world', 'play', 'football', 'match'] 

Document (CNN) -- Manchester United and their city rivals Manchester City both made embarrassing exits from the European Champions League on Wednesday.

United were beaten 2-1 in Basel to leave the Swiss side and Benfica as the qualifiers from Group C, while City beat Bayern Munich 2-0 at the Etihad Stadium but went out as Napoli won 2-0 at Villarreal.

It left Bayern and the Italians to go through from Group A to complete a chastening first experience of Europe's top club competition for Robert Mancini's expensively assembled side.

But it was the exit of last year's runners-up United which was the biggest upset, needing only a draw to ensure their passage to the last 16, goals from Marco Streller and Alexander Frei sent them crashing.

Real Madrid to topple Barca in 'El Clasico' clash?

Phil Jones pulled one back in the 89th minute to set up a frantic finale but they could not forc

In [ ]:
# Consolidated dictionary of topic numbers to text

mapped_dictionary_document={}

for i in range(len(topics_by_documnent_numbers)):
    mapped_dictionary_document[i]=get_document2topic(i)

In [53]:
#corpus[78000][:1000]

'(CNN)Well, the thought experiments are finally over. Hillary Clinton\'s online declaration for president means the focus will now shift to the campaign and to what kind of president she might be.\n\nAnd nowhere will the speculation be greater than in the area of foreign policy, certain to be a major issue in the upcoming campaign.\n\nShould she win, Clinton will add to her potential "firsts": first woman president; the first president who had been a first lady. There\'s another one, too: the first secretary of state to become president since James Buchanan.\n\nOnly a handful of the nation\'s top diplomats have gone to the White House (Thomas Jefferson; James Madison; James Monroe; John Quincy Adams; Martin Van Buren; and Buchanan). And none in the 20th century, a curious fact that suggests politics and statecraft are quite different animals. Too few of our secretaries of state have had the necessary experience of elected office or the desire, ambition and temperament to compete for th

## Visualization

In [41]:

import pyLDAvis.gensim

In [42]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda, cv_2corpus,ld_words2idf )

/home/ubuntu/anaconda/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=           Freq  cluster  topics         x         y
topic                                               
18     7.938478        1       1 -0.156258 -0.053360
7      5.816388        1       2  0.003758  0.059117
23     5.615049        1       3  0.052993  0.047702
4      5.278841        1       4  0.198383  0.063322
31     4.349673        1       5  0.145139 -0.074118
13     4.347507        1       6  0.039853 -0.140367
2      4.176764        1       7 -0.115448 -0.073216
8      4.070444        1       8 -0.132340 -0.123398
26     4.016914        1       9  0.020430 -0.195575
30     4.004967        1      10  0.131216  0.125149
9      3.878934        1      11 -0.005519  0.015428
17     3.703269        1      12 -0.173738  0.153496
11     3.268021        1      13  0.032663  0.089612
6      3.134234        1      14  0.128425  0.133367
20     3.112955        1      15  0.024272 -0.069242
32     2.928497        1      16  0.013858 -0.060797
29     2.679519        1      17  0.034214 -0.107566
0      2.544307        1      18 -0.153940 -0.037192
12     2.478858        1      19  0.066755 -0.061696
3      2.466159        1      20 -0.135152 -0.040255
25     2.227072        1      21 -0.075447 -0.026997
27     2.144117        1      22 -0.027622 -0.131824
16     2.043852        1      23 -0.064336  0.020901
10     1.975464        1      24 -0.175150  0.092727
22     1.896895        1      25  0.162994  0.114073
1      1.716882        1      26 -0.153775  0.182355
15     1.702634        1      27  0.047717 -0.004390
21     1.581321        1      28  0.112871 -0.071308
24     1.195180        1      29  0.075931 -0.050361
19     0.967556        1      30  0.130549  0.061447
14     0.951069        1      31 -0.183972  0.097937
5      0.935439        1      32  0.139385  0.036109
28     0.852739        1      33 -0.008708  0.028918, topic_info=      Category           Freq          Term          Total  loglift  logprob
term                                                                        
152    Default   58983.000000         obama   58983.000000  30.0000  30.0000
1182   Default   75905.000000        police   75905.000000  29.0000  29.0000
94     Default   77334.000000     president   77334.000000  28.0000  28.0000
2748   Default   39863.000000        school   39863.000000  27.0000  27.0000
5627   Default   39533.000000         women   39533.000000  26.0000  26.0000
636    Default   58904.000000           win   58904.000000  25.0000  25.0000
5073   Default   45554.000000         court   45554.000000  24.0000  24.0000
742    Default   83846.000000         world   83846.000000  23.0000  23.0000
324    Default   73336.000000    government   73336.000000  22.0000  22.0000
677    Default   39719.000000      security   39719.000000  21.0000  21.0000
6871   Default   20319.000000          film   20319.000000  20.0000  20.0000
683    Default   33406.000000        health   33406.000000  19.0000  19.0000
1967   Default   39178.000000      children   39178.000000  18.0000  18.0000
343    Default   49036.000000        family   49036.000000  17.0000  17.0000
75     Default   45117.000000        attack   45117.000000  16.0000  16.0000
4735   Default   18752.000000        flight   18752.000000  15.0000  15.0000
1588   Default   21960.000000          race   21960.000000  14.0000  14.0000
6353   Default   21663.000000          drug   21663.000000  13.0000  13.0000
38     Default  125856.000000         state  125856.000000  12.0000  12.0000
8694   Default   26421.000000         china   26421.000000  11.0000  11.0000
93     Default   96078.000000        report   96078.000000  10.0000  10.0000
4361   Default   15394.000000        russia   15394.000000   9.0000   9.0000
8396   Default   17963.000000          iran   17963.000000   8.0000   8.0000
2725   Default   15810.000000      students   15810.000000   7.0000   7.0000
1815   Default   41815.000000          play   41815.000000   6.0000   6.0000
7897   Default   13750.

In [46]:
top_topics = lda.top_topics(cv_2corpus, topn=20)

2018-03-04 01:39:21,334 : INFO : CorpusAccumulator accumulated stats from 1000 documents
2018-03-04 01:39:21,436 : INFO : CorpusAccumulator accumulated stats from 2000 documents
2018-03-04 01:39:21,532 : INFO : CorpusAccumulator accumulated stats from 3000 documents
2018-03-04 01:39:21,631 : INFO : CorpusAccumulator accumulated stats from 4000 documents
2018-03-04 01:39:21,732 : INFO : CorpusAccumulator accumulated stats from 5000 documents
2018-03-04 01:39:21,828 : INFO : CorpusAccumulator accumulated stats from 6000 documents
2018-03-04 01:39:21,923 : INFO : CorpusAccumulator accumulated stats from 7000 documents
2018-03-04 01:39:22,020 : INFO : CorpusAccumulator accumulated stats from 8000 documents
2018-03-04 01:39:22,119 : INFO : CorpusAccumulator accumulated stats from 9000 documents
2018-03-04 01:39:22,216 : INFO : CorpusAccumulator accumulated stats from 10000 documents
2018-03-04 01:39:22,316 : INFO : CorpusAccumulator accumulated stats from 11000 documents
2018-03-04 01:39:22